# Hangman challenge using Trie structure

In [ ]:
import pandas as pd
import numpy as np
import os
import random
import string
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from itertools import combinations
import json
import requests
import random
import string
import time
import re
import collections
from collections import Counter

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
def func(new_dictionary):
    """
    Count the occurrences of each letter in the new dictionary.

    Parameters:
    - new_dictionary (list): List of possible words.

    Returns:
    - collections.Counter: Count of occurrences for each letter.
    """
    dictx = collections.Counter()
    for words in new_dictionary:
        temp = collections.Counter(words)
        for i in temp:
            temp[i] = 1
            dictx = dictx + temp
    return dictx

In [ ]:
# Construct Trie structure to store patterns in dictionary
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False
        self.frequency = 0  # Frequency count indicating the number of words passing through the node


class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.trie_count = 0  # Initialize Trie count

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
                self.trie_count += 1  # Increment Trie count when a new Trie is created
            node = node.children[char]
            node.frequency += 1  # Increment frequency count for the node
        node.is_end_of_word = True

    def construct_from_dictionary(self, dictionary):
        for word in dictionary:
            for i in range(len(word)):
                self.insert(word[i:])  # Create Trie structures starting from each letter in the word


    def find_possible_options(self, masked_word, excluded_letters=None):
        if excluded_letters is None:
            excluded_letters = set()

        node = self.root
        for char in masked_word:
            if char != '_' and char in node.children:
                node = node.children[char]
            elif char == '_':
                break
            else:
                return []  # No words matching the pattern

        options = []
        self.traverse_with_frequency(node, masked_word, '', options, excluded_letters)

        return options

    def traverse_with_frequency(self, node, remaining_pattern, current_word, options, excluded_letters):
        if not remaining_pattern:
            if node.is_end_of_word:
                options.append((current_word, node.frequency))  # Include frequency in options
            return

        for char, child_node in node.children.items():
            if remaining_pattern[0] == '_':
                if char not in excluded_letters:
                    # Adjust frequency weight based on the number of children and their frequencies
                    child_frequency = max(1, child_node.frequency)
                    self.traverse_with_frequency(child_node, remaining_pattern[1:], current_word + char, options, excluded_letters)
            elif remaining_pattern[0] == char:
                self.traverse_with_frequency(child_node, remaining_pattern[1:], current_word + char, options, excluded_letters)

    def get_trie_count(self):
        """
        Get the total number of Trie structures created.
        """
        return self.trie_count

In [ ]:
# Construct Trie to detect prefix.
class PrefixTrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_prefix = False
        self.frequency = 0


class PrefixTrie:
    def __init__(self):
        self.root = PrefixTrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = PrefixTrieNode()
            node = node.children[char]
        node.is_end_of_prefix = True
        node.frequency += 1

    def construct_from_dictionary(self, dictionary, threshold):
        for word in dictionary:
            for i in range(3, min(6, len(word) + 1)):
                prefix = word[:i]
                self.insert(prefix)
        self.prune_tree(threshold)

    def prune_tree(self, threshold):
        stack = [(self.root, '')]
        while stack:
            node, prefix = stack.pop()
            if node.is_end_of_prefix and node.frequency < threshold:
                del node
                continue
            for char, child_node in node.children.items():
                stack.append((child_node, prefix + char))

    def get_prefixes_with_frequency(self):
        prefixes_frequency = {}
        stack = [(self.root, '')]
        while stack:
            node, prefix = stack.pop()
            if node.is_end_of_prefix:
                prefixes_frequency[prefix] = node.frequency
            for char, child_node in node.children.items():
                stack.append((child_node, prefix + char))
        return prefixes_frequency

def find_prefix_from_masked_word(masked_word, potential_prefixes):
    matched_prefix = []
    # Iterate through each potential prefix
    for prefix in potential_prefixes:
        # Check if the length of the masked word is greater than or equal to the length of the potential prefix
        if len(masked_word) >= len(prefix):
            # Iterate through each character in the masked word and compare it with the corresponding character in the potential prefix
            match = True
            for i in range(len(prefix)):
                if masked_word[i] != "_" and masked_word[i] != prefix[i]:
                    match = False
                    break
            if match:
                matched_prefix.append(prefix)  # Return the matching prefix
    return matched_prefix  # If no matching prefix is found, return None

In [ ]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        self.full_dictionary_location = "/content/drive/MyDrive/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(self.full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.tries_remains  = 6

        self.fulldata_df = self.read_data()
        self.hangman_trie_full = Trie()
        self.hangman_trie_full.construct_from_dictionary(self.fulldata_df[0])

        self.prefix_list = self.prefix_finding()
        self.freq_threshold = 200

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link


    def read_data(self):
        # Function to read data from a file
        with open(self.full_dictionary_location, "r") as f:
            df = f.read()
        x = pd.DataFrame(df.split('\n'))
        return x

    def prefix_finding (self):
        # finding the prefix list
        threshold = 1000
        prefix_trie = PrefixTrie()
        prefix_trie.construct_from_dictionary(self.fulldata_df[0], threshold)
        prefixes_frequency = prefix_trie.get_prefixes_with_frequency()
        # Sort the dictionary by frequency
        sorted_prefixes_frequency = dict(sorted(prefixes_frequency.items(), key=lambda item: item[1], reverse=True))
        prefix_list = [key for key,value in sorted_prefixes_frequency.items() if value > threshold]
        return prefix_list


    def generate_ngrams(self, masked_word, n):
        # constrcting ngram from the mask word
        ngrams = []
        for i in range(len(masked_word) - n + 1):
            window = masked_word[i:i+n]
            if '_' in window and any(c.isalpha() for c in window):
                ngrams.append(window)
        return ngrams

    def possible_letters(self, masked_word, excluded_letter=None):
        # Initialize an empty list to store possible letter options
        possible_options = []
        # Get the length of the masked word
        ngram_len = len(masked_word)
        # Initialize the sum of frequencies of found patterns
        freq_sum = 0

        # Iterate until the frequency threshold is reached or until ngram_len reaches 1
        while freq_sum < self.freq_threshold:
            if ngram_len == 1:
                break

            # Generate possible n-grams from the masked word
            possible_ngrams = self.generate_ngrams(masked_word, ngram_len)

            # Iterate over each n-gram and find possible options using the Trie
            for ngram in possible_ngrams:
                current_option = self.hangman_trie_full.find_possible_options(ngram, excluded_letter)
                possible_options.extend(current_option)

            # Decrease the n-gram length for the next iteration
            ngram_len -= 1

            # Calculate the sum of frequencies of found options
            freq_sum = sum([freq for text, freq in possible_options])

        # Print the sum of frequencies and the final n-gram length used

        # Weight the possible options based on their frequencies
        possible_options_weighted = [text * freq for text, freq in possible_options]

        # Concatenate all words into a single string
        all_letters = ''.join(possible_options_weighted)

        # Count the occurrence of each letter
        letter_counts = Counter(all_letters)

        # Sort the letters based on their counts in descending order
        sorted_alphabets_withcount = sorted(letter_counts.items(), key=lambda x: x[1], reverse=True)

        # Extract the sorted letters from the sorted_alphabets_withcount list
        sorted_alphabets = [alphabet for alphabet, count in sorted_alphabets_withcount]

        # Return the sorted list of letters
        return sorted_alphabets


    def guess(self, word):
        # word input example: "_ p p _ e "

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")

        # find length of passed word
        len_word = len(clean_word)

        # remaing spaces
        remaining_spaces = clean_word.count('.')

        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []

        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue

            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word, dict_word):
                new_dictionary.append(dict_word)

        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary

        # add the incorrect gussed letters
        if self.guessed_letters and self.guessed_letters[-1] not in clean_word:
            self.excluded_letter.append(self.guessed_letters[-1])


        # start the guess letter
        guess_letter = '!'

        # if we have not yet guessed at least 2, start with most common letters,
        # in the dictionary of the words with the same length
        if (len_word - remaining_spaces) <= 2 or self.tries_remains > 4:
            new_dict_string = "".join(new_dictionary)
            # return most frequently occurring letter in all possible words that hasn't been guessed yet
            c = collections.Counter(new_dict_string)
            sorted_letter_count = c.most_common()
            for letter,_ in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        # now we have at least two letters:
        if guess_letter == '!':
            predict_letters = self.possible_letters(word[::2])
            for letter in predict_letters:
                # check if the prediction is alphabet and it is not already suggested
                if letter.isalpha() and letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        # check for prefix:
        # if there is only 2 left and 3 space left and there is at least one missing in first 4 letter
        if self.tries_remains <= 2 and remaining_spaces < 4 and any(letter == "." for letter in clean_word[:4]):
            found_prefix = find_prefix_from_masked_word(word[::2], self.prefix_list)
            full_dict_string = "".join(found_prefix)
            # return most frequently occurring letter in all possible words that hasn't been guessed yet
            c = collections.Counter(full_dict_string)
            sorted_letter_count = c.most_common()
            for letter,_ in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        # if there was no match: based on words with the same pattern:
        if guess_letter == '!':
            new_dict_string = "".join(new_dictionary)
            # return most frequently occurring letter in all possible words that hasn't been guessed yet
            c = collections.Counter(new_dict_string)
            sorted_letter_count = c.most_common()
            for letter,_ in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break


        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,_ in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.excluded_letter = []


        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            self.tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, self.tries_remains, word))
            while self.tries_remains > 0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                self.tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [ ]:
api.start_game(practice=1, verbose=True)

Successfully start a new game! Game ID: b14f3f7e2972. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'b14f3f7e2972', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': 'b14f3f7e2972', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ i _ _ _ _ _ i _ _ '}
Guessing letter: n
Sever response: {'game_id': 'b14f3f7e2972', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ i n _ _ _ _ i n _ '}
Guessing letter: g
Sever response: {'game_id': 'b14f3f7e2972', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ i n _ _ _ _ i n g '}
Guessing letter: r
Sever response: {'game_id': 'b14f3f7e2972', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ i n _ _ _ _ i n g '}
Guessing letter: a
Sever response: {'game_id': 'b14f3f7e2972', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ a i n _ _ _ _ i n g '}
Guessing letter: t
Sever response: {'game_id': 'b

True